In [1]:
# import requests
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import cv2
import os
from pathlib import Path
import numpy as np
import math


image_name = "file"
image_display_name = "display"
image_folder = "folder"
FOLDERLABEL = '-------FOLDERS-------'
FILESLABEL = '-------FILES-------'
actual_length, actual_pixel = 0.0, 0.0
polygon_pixel_area = 0.0
x_pixel, y_pixel = [], []

drawing = False # true if mouse is pressed
pt1_x , pt1_y = None , None

cwd = Path(os.getcwd())

def on_button_clicked1(b):#b refers to button
    global image_name
    response = str(w_text1.value + '.png')
    w_output.clear_output()
    with w_output:
        image_name = response
    
def on_button_clicked2(b): #b refers to button
    global image_name, image_folder
    #w_text.value is the text I put in the button
    response = str(w_text2.value)
    image_folder = response
    prefix = response + '\\'
    w_output.clear_output()
    
    video = cv2.VideoCapture(0)
    with w_output:
        while video.isOpened():
            check, frame = video.read()
            #gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            cv2.imshow("Capturing",frame)
            if cv2.waitKey(1) &0XFF == ord('x'):
                 break
        showPic = cv2.imwrite(prefix + image_name, frame)
        video.release()
        cv2.destroyAllWindows()
        
def image_button_select(b):
    global image_display_name
    response = str(im_text1.value + '.png')
    im_output.clear_output()
    with im_output:
        image_display_name = response

          
        
def image_button_display(b):
    def line_drawing(event,x,y,flags,param):
        global pt1_x,pt1_y,drawing, x_pixel, y_pixel

        if event==cv2.EVENT_LBUTTONDOWN:
            drawing=True
            pt1_x,pt1_y=x,y
            x_pixel.append(x)
            y_pixel.append(y)
            
            cv2.circle(img, (x,y), 0, color=(0,0,255), thickness = 3)

        #elif event==cv2.EVENT_MOUSEMOVE:
        #    if drawing==True:
        #        cv2.line(img,(pt1_x,pt1_y),(x,y),color=(255,255,255),thickness=3)
        #        pt1_x,pt1_y=x,y
        elif event==cv2.EVENT_LBUTTONUP:
            drawing=False
            cv2.line(img,(pt1_x,pt1_y),(x,y),color=(0,0,200),thickness=3)
            # line is red color
            cv2.circle(img, (x,y), 0, color=(0,0,255), thickness = 3)
            
    global image_display_name
    response = str(image_folder + "\\" + image_display_name)
    im_output.clear_output()
    with im_output:
        img = cv2.imread(response)
        cv2.namedWindow('Draw a polygon')
        cv2.setMouseCallback('Draw a polygon',line_drawing)
        
        while(1):
            cv2.imshow('Draw a polygon',img)
            if cv2.waitKey(1)&0xFF == ord('x'):
                break

        cv2.destroyAllWindows()
        
def write_actual_length(b):
    global actual_length
    response = im_text2.value
    im_output.clear_output()
    with im_output:
        actual_length = float(response)

def draw_line_button(b):#b refers to button
    def line_drawing(event,x,y,flags,param):
        global pt1_x,pt1_y,drawing, actual_pixel

        if event==cv2.EVENT_LBUTTONDOWN:
            drawing=True
            pt1_x,pt1_y=x,y
            cv2.circle(img, (x,y), 0, color=(0,0,255), thickness = 3)

        elif event==cv2.EVENT_LBUTTONUP:
            drawing=False
            cv2.line(img,(pt1_x,pt1_y),(x,y),color=(0,0,200),thickness=3)
            # line is red color
            cv2.circle(img, (x,y), 0, color=(0,0,255), thickness = 3)
            actual_pixel = math.sqrt((x-pt1_x)*(x-pt1_x) + (y-pt1_y)*(y-pt1_y))
            
    global image_display_name, drawing
    response = str(image_folder + "\\" + image_display_name)
    im_output.clear_output()
    with im_output:
        img = cv2.imread(response)

        cv2.namedWindow('Draw a line')
        cv2.setMouseCallback('Draw a line',line_drawing)
        
        while(1):
            cv2.imshow('Draw a line',img)
            if cv2.waitKey(1)&0xFF == ord('x'):
                break

        cv2.destroyAllWindows()  

def display_float(b):
    global actual_length, actual_pixel, x_pixel, y_pixel
    im_output.clear_output()
    with im_output:
        display(widgets.FloatText(
            value=PolyArea(x_pixel, y_pixel) * math.pow((actual_length/ actual_pixel), 2),
            description='Actual area:',
            disabled=False
        ))

    
#############################################################################################

#w_header = widgets.HTML('<h2><b>Choose an image folder</b></h2>')
w_text1 = widgets.Textarea(placeholder='Choose',layout = widgets.Layout(width = '30%'))

w_button1 = widgets.Button(description = 'Input ID to store')
w_button1.on_click(on_button_clicked1)

w_text2 = widgets.Textarea(placeholder='Choose',layout = widgets.Layout(width = '80%'))

w_button2 = widgets.Button(description = 'Take photo & Save')
w_button2.on_click(on_button_clicked2)

w_output = widgets.Output()

w_ui = widgets.VBox([w_text1, w_button1, w_text2, w_button2, w_output], layout=widgets.Layout(align_items='center'))

#############################################################################################

im_text1 = widgets.Textarea(placeholder='Choose',layout = widgets.Layout(width = '30%'))

im_button1 = widgets.Button(description = 'Input ID to display')
im_button1.on_click(image_button_select)

im_text2 = widgets.Textarea(placeholder='Enter length in cm',layout = widgets.Layout(width = '80%'))
im_button2 = widgets.Button(description = 'Enter length in cm')
im_button2.on_click(write_actual_length)

im_button3 = widgets.Button(description = 'Draw a line')
im_button3.on_click(draw_line_button)

im_button4 = widgets.Button(description = 'Draw a polygon')
im_button4.on_click(image_button_display)

im_button5 = widgets.Button(description = 'Display parameters')
im_button5.on_click(display_float)

im_output = widgets.Output()

im_ui = widgets.VBox([im_text1, im_button1, im_text2, im_button2, im_button3, im_button4, im_button5, im_output], layout=widgets.Layout(align_items='center'))

#############################################################################################

def PolyArea(x,y):
    x, y = np.array(x), np.array(y)
    
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))



#############################################################################################




def get_folder_contents(folder):
    folder = Path(folder)
    folders = [item.name for item in folder.iterdir() if item.is_dir() and not item.name.startswith('.')]
    files = [item.name for item in folder.iterdir() if item.is_file() and not item.name.startswith('.')]
    return [FOLDERLABEL] + sorted(folders) + [FILESLABEL] + sorted(files)

def selecting(value):
    global image_value
    if value['new'] and value['new'] not in [FOLDERLABEL, FILESLABEL]:
        path = Path(address_field.value)
        newpath = path / value['new']
        if newpath.is_dir():
            go_to_address(newpath)
        elif newpath.is_file():
            #if newpath.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                # Try displaying the image here
            pass

def parent(value):
    new = Path(address_field.value).parent
    go_to_address(new)
    
def go_to_address(address):
    address = Path(address)
    if address.is_dir():
        address_field.value = str(address)
        select.unobserve(selecting, names='value')
        select.options = get_folder_contents(folder=address)
        select.observe(selecting, names='value')
        select.value = None

def newaddress(value):
    go_to_address(address_field.value)
        

address_field = widgets.Text(value=str(cwd))
up_button = widgets.Button(description='Up')
select = widgets.Select(options=get_folder_contents(cwd), rows=20, value=None)
select.observe(selecting, names='value')
up_button.on_click(parent)
address_field.on_submit(newaddress)
box = widgets.HBox([widgets.VBox([address_field, select]), widgets.VBox([up_button]), w_ui, im_ui])


box